In [ ]:
from load_docs import load_documents,chunk_documents
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import HuggingFaceHub
import os
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFaceEndpoint
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

In [3]:
# 1. Load PDF
documents = load_documents("docs/na_tesla_service_plan_agreement_ot.pdf")

Ignoring wrong pointing object 8 0 (offset 0)


In [4]:
# 2. Split into chunks
chunks = chunk_documents(documents)

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en")
vectorstore = Chroma.from_documents(chunks, embeddings)

/var/folders/y6/k28bmfyn2zvc2txjktsxbxhw0000gn/T/ipykernel_83620/1600519414.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en")
/Users/tomthomas/Downloads/RAG Learning/Vanilla RAG/vanillarag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    device=-1   # CPU, or 0 for GPU
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)


Device set to use cpu
/var/folders/y6/k28bmfyn2zvc2txjktsxbxhw0000gn/T/ipykernel_83620/3045912102.py:7: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [14]:
# 4. Create QA chain
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
# Point to your deployed Inference Endpoint)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [15]:
# 5. Ask a question
query = "Summarize the document in 3 points"
answer = qa_chain.run(query)

Token indices sequence length is longer than the specified maximum sequence length for this model (1857 > 512). Running this sequence through the model will result in indexing errors


In [16]:
print("Q:", query)
print("A:", answer)

Q: Summarize the document in 3 points
A: TSPA012013 TSPA012013 E. Agreement Period This Agreement is an application for coverage under an Agreement. Sections B through K shall survive any termination or expiration of this Agreement. F. Territory This Agreement applies to a Vehicle sold by Tesla in the Tesla North America Warranty Region and transported or driven only in the Tesla North America Warranty Region. For purposes of this Agreement, the Tesla North America Warranty Region is defined as all 50 states of the United States of America, the District of Columbia, and all 13 provinces and territories of Canada. If Your Vehicle was sold, transported or driven outside the Tesla North America Warranty Region, no warranties or coverage, including this Agreement, will apply. Tesla will not pay for or reimburse You for services that are performed by any party other than a Tesla Authorized Service Center or Tesla Ranger. Tesla hereby disclaims any and all indirect, incidental, special and c